### Question 3:
Program types and hours correlated with performance/scores.

Author: Thomas

Updates:
* Last updated on March 23, 2020
* Updated on March 21, 2020


In [1]:
# import dependencies
import os
import pandas as pd
import numpy as np
import json

In [2]:
# CSV Files
math_results = 'data/2013-2019_school_math_results.csv'
ela_results = 'data/2013-2019_school_ela_results.csv'
arts_2017 = 'data/2016-2017_Arts_Survey_Data.csv'
arts_2018 = 'data/2017-2018_Arts_Survey_Data.csv'
arts_2019 = 'data/2018-2019_Arts_Survey_Data.csv'

In [3]:
math_df = pd.read_csv(math_results, encoding='utf-8', low_memory=False)
math_df.head()
print(f'Math Results Rows: {math_df.shape}')
print(list(math_df.columns))
print()
print(math_df['Year'].unique())

Math Results Rows: (32606, 18)
['Unnamed: 0', 'DBN', 'School Name', 'Grade', 'Year', 'Category', 'Number Tested', 'Mean Scale Score', '# Level 1', '% Level 1', '# Level 2', '% Level 2', '# Level 3', '% Level 3', '# Level 4', '% Level 4', '# Level 3+4', '% Level 3+4']

[2013 2014 2015 2016 2017 2018 2019]


In [4]:
# drop unneeded columns
math_df = math_df.drop(labels='Unnamed: 0', axis=1)

In [5]:
math_df.head(1)

,DBN,School Name,Grade,Year,Category,Number Tested,Mean Scale Score,# Level 1,% Level 1,# Level 2,% Level 2,# Level 3,% Level 3,# Level 4,% Level 4,# Level 3+4,% Level 3+4
0,01M015,P.S. 015 ROBERTO CLEMENTE,3,2013,All Students,27,277.777771,16,59.25925827,11,40.74074173,0,0,0,0,0,0


In [6]:
ela_df = pd.read_csv(ela_results, encoding='utf-8', low_memory=False)
ela_df.head()
print(f'ELA Results Rows: {ela_df.shape}')
print(list(ela_df.columns))
print()
print(ela_df['Year'].unique())

ELA Results Rows: (32826, 18)
['Unnamed: 0', 'DBN', 'School Name', 'Grade', 'Year', 'Category', 'Number Tested', 'Mean Scale Score', '# Level 1', '% Level 1', '# Level 2', '% Level 2', '# Level 3', '% Level 3', '# Level 4', '% Level 4', '# Level 3+4', '% Level 3+4']

[2013 2014 2015 2016 2017 2018 2019]


In [7]:
# drop unneeded columns
ela_df = ela_df.drop(labels='Unnamed: 0', axis=1)

In [8]:
ela_df.head(1)

,DBN,School Name,Grade,Year,Category,Number Tested,Mean Scale Score,# Level 1,% Level 1,# Level 2,% Level 2,# Level 3,% Level 3,# Level 4,% Level 4,# Level 3+4,% Level 3+4
0,01M015,P.S. 015 ROBERTO CLEMENTE,3,2013,All Students,27,289.2962952,14,51.85185242,11,40.74074173,2,7.407407284,0,0,2,7.407407284


In [9]:
arts17_df = pd.read_csv(arts_2017, encoding='utf-8', low_memory=False)
arts18_df = pd.read_csv(arts_2018, encoding='utf-8', low_memory=False)
arts19_df = pd.read_csv(arts_2019, encoding='utf-8', low_memory=False)

In [10]:
# Add missing column `Year` to survey files
arts17_df['Year'] = 2017
arts18_df['Year'] = 2018
arts19_df['Year'] = 2019

In [11]:
# rename the school DBN column to match in all dataframes
updCol = { 'Q0_DBN': 'DBN' }
arts17_df = arts17_df.rename(columns=updCol)
arts18_df = arts18_df.rename(columns=updCol)
arts19_df = arts19_df.rename(columns=updCol)

In [38]:
#list(arts19_df.columns)
#arts19_df['Q3'].head(5)
arts19_df.shape

(1546, 1182)

In [ ]:
### 2018-2019 Arts Survey Questions

### Arts Supervisors
print({
    'Q3_1': 'FT_Arts_Only',
    'Q3_2': 'FT_Arts_Plus',
    'Q3_3': 'PT_Arts_Plus',
    'Q3_4': 'No_Supervisor',
})
#    'Q4': 'IsSuperArtsCert',
print({
    'Q4_1': 'Certified',
    'Q4_2': 'NotCertified',
})
#    'Q5': 'IsSuperAdminCert'
print({
    'Q5_1': 'Certified',
    'Q5_2': 'NotCertified',
})

### Number of Part-Time Certified Art Teachers
print({ 
    'Q6_1': 'NumDanceTchr',
    'Q6_2': 'NumMusicTchr',
    'Q6_3': 'NumThetrTchr',
    'Q6_4': 'NumVArtsTchr',
 })

 ### Arts Offered (District 75)
 print({
     'Q10_1': 'Dance',
     'Q10_2': 'Film',
     'Q10_3': 'Music',
     'Q10_4': 'Theater',
     'Q10_5': 'VisualArts'
 })

 ### Pre-K Arts
 print({
     'Q11_1': 'Dance',
     'Q11_2': 'Music',
     'Q11_3': 'Theater',
     'Q11_4': 'VisualArts'
 })

 ### Pre-K Arts Hours (by Cultural Arts Organizations)
  print({
     'Q12_1': 'Dance',
     'Q12_2': 'Music',
     'Q12_3': 'Theater',
     'Q12_4': 'VisualArts'
 })

 ### First Grade Arts Hours (by Cultural Arts Organizations)
  print({
     'Q13_1': 'Dance',
     'Q13_2': 'Music',
     'Q13_3': 'Theater',
     'Q13_4': 'VisualArts'
 })

  ### Second Grade Arts Hours (by Cultural Arts Organizations)
  print({
     'Q14_1': 'Dance',
     'Q14_2': 'Music',
     'Q14_3': 'Theater',
     'Q14_4': 'VisualArts'
 })

  ### Third Grade Arts Hours (by Cultural Arts Organizations)
  print({
     'Q15_1': 'Dance',
     'Q15_2': 'Music',
     'Q15_3': 'Theater',
     'Q15_4': 'VisualArts'
 })
  ### Fourth Grade Arts Hours (by Cultural Arts Organizations)
  print({
     'Q16_1': 'Dance',
     'Q16_2': 'Music',
     'Q16_3': 'Theater',
     'Q16_4': 'VisualArts'
 })
  ### Fifth Grade Arts Hours (by Cultural Arts Organizations)
  print({
     'Q17_1': 'Dance',
     'Q17_2': 'Music',
     'Q17_3': 'Theater',
     'Q17_4': 'VisualArts'
 })

 ### Sixth Grade Arts Hours (by Cultural Arts Organizations)
  print({
     'Q19_1': 'Dance',
     'Q19_2': 'Music',
     'Q19_3': 'Theater',
     'Q19_4': 'VisualArts'
 })
  
  ### Parental Involvement
  ### ['Attending School Events', 'Volunterring in Classrooms', 'Donating Materials', 'Other']
  print({
      'Q35': 'ParentalInvolve'
  })

  ### Student Participation
  ### Site: ['School', 'Outside']
  print({
      'Q36_1': 'ArtExibits'
      'Q36_2': 'Concerts',
      'Q36_3': 'Dance Performances',
      'Q36_4': 'Films',
      'Q36_5': 'Theater Performances',
      'Q36_6': 'Other'
  })

  ### Arts/Cultural Organizations
  ### Question 40

  ### Arts Education Quality Goals
  ### Question 41

In [56]:
# sum up all Kindergarten Hours
gk = { 
    'dance': ['DBN'],
    'music': ['DBN'],
    'theater': ['DBN'],
    'visual': ['DBN']
}

for col in arts19_df.columns:
    if ('Q12' in col) & ('R1' in col):
        gk['dance'].append(col)
    elif ('Q12' in col) & ('R2' in col):
        gk['music'].append(col)
    elif ('Q12' in col) & ('R3' in col):
        gk['theater'].append(col)
    elif ('Q12' in col) & ('R4' in col):
        gk['visual'].append(col)

arts19_df['GK_Dance_Hrs'] = arts19_df[gk['dance']].fillna(0).sum(axis=1)
arts19_df['GK_Music_Hrs'] = arts19_df[gk['music']].fillna(0).sum(axis=1)
arts19_df['GK_Theater_Hrs'] = arts19_df[gk['theater']].fillna(0).sum(axis=1)
arts19_df['GK_Visual_Hrs'] = arts19_df[gk['visual']].fillna(0).sum(axis=1)

arts19_df[['DBN','GK_Dance_Hrs','GK_Music_Hrs','GK_Theater_Hrs','GK_Visual_Hrs']].head(2)

,DBN,GK_Dance_Hrs,GK_Music_Hrs,GK_Theater_Hrs,GK_Visual_Hrs
0,01M015,18.0,30.0,0.0,29.0
1,01M019,63.0,13.0,60.0,63.0


In [40]:
# sum up all First Grade Hours
g1 = { 
    'dance': ['DBN'],
    'music': ['DBN'],
    'theater': ['DBN'],
    'visual': ['DBN']
}

for col in arts19_df.columns:
    if ('Q13' in col) & ('R1' in col):
        g1['dance'].append(col)
    elif ('Q13' in col) & ('R2' in col):
        g1['music'].append(col)
    elif ('Q13' in col) & ('R3' in col):
        g1['theater'].append(col)
    elif ('Q13' in col) & ('R4' in col):
        g1['visual'].append(col)
        
arts19_df['G1_Dance_Hrs'] = arts19_df[g1['dance']].fillna(0).sum(axis=1)
arts19_df['G1_Music_Hrs'] = arts19_df[g1['music']].fillna(0).sum(axis=1)
arts19_df['G1_Theater_Hrs'] = arts19_df[g1['theater']].fillna(0).sum(axis=1)
arts19_df['G1_Visual_Hrs'] = arts19_df[g1['visual']].fillna(0).sum(axis=1)

arts19_df[['DBN','G1_Dance_Hrs','G1_Music_Hrs','G1_Theater_Hrs','G1_Visual_Hrs']].head(2)

,DBN,G1_Dance_Hrs,G1_Music_Hrs,G1_Theater_Hrs,G1_Visual_Hrs
0,01M015,36.0,60.0,0.0,58.0
1,01M019,204.0,69.0,60.0,204.0


In [43]:
# sum up all Second Grade Hours
g2 = { 
    'dance': ['DBN'],
    'music': ['DBN'],
    'theater': ['DBN'],
    'visual': ['DBN']
}

for col in arts19_df.columns:
    if ('Q14' in col) & ('R1' in col):
        g2['dance'].append(col)
    elif ('Q14' in col) & ('R2' in col):
        g2['music'].append(col)
    elif ('Q14' in col) & ('R3' in col):
        g2['theater'].append(col)
    elif ('Q14' in col) & ('R4' in col):
        g2['visual'].append(col)
        
arts19_df['G2_Dance_Hrs'] = arts19_df[g2['dance']].fillna(0).sum(axis=1)
arts19_df['G2_Music_Hrs'] = arts19_df[g2['music']].fillna(0).sum(axis=1)
arts19_df['G2_Theater_Hrs'] = arts19_df[g2['theater']].fillna(0).sum(axis=1)
arts19_df['G2_Visual_Hrs'] = arts19_df[g2['visual']].fillna(0).sum(axis=1)

arts19_df[['DBN','G2_Dance_Hrs','G2_Music_Hrs','G2_Theater_Hrs','G2_Visual_Hrs']].head(2)

,DBN,G2_Dance_Hrs,G2_Music_Hrs,G2_Theater_Hrs,G2_Visual_Hrs
0,01M015,0.0,60.0,22.0,58.0
1,01M019,204.0,69.0,120.0,204.0


In [44]:
# sum up all Third Grade Hours
g3 = { 
    'dance': ['DBN'],
    'music': ['DBN'],
    'theater': ['DBN'],
    'visual': ['DBN']
}

for col in arts19_df.columns:
    if ('Q15' in col) & ('R1' in col):
        g3['dance'].append(col)
    elif ('Q15' in col) & ('R2' in col):
        g3['music'].append(col)
    elif ('Q15' in col) & ('R3' in col):
        g3['theater'].append(col)
    elif ('Q15' in col) & ('R4' in col):
        g3['visual'].append(col)
        
arts19_df['G3_Dance_Hrs'] = arts19_df[g3['dance']].fillna(0).sum(axis=1)
arts19_df['G3_Music_Hrs'] = arts19_df[g3['music']].fillna(0).sum(axis=1)
arts19_df['G3_Theater_Hrs'] = arts19_df[g3['theater']].fillna(0).sum(axis=1)
arts19_df['G3_Visual_Hrs'] = arts19_df[g3['visual']].fillna(0).sum(axis=1)

arts19_df[['DBN','G3_Dance_Hrs','G3_Music_Hrs','G3_Theater_Hrs','G3_Visual_Hrs']].head(2)

,DBN,G3_Dance_Hrs,G3_Music_Hrs,G3_Theater_Hrs,G3_Visual_Hrs
0,01M015,0.0,60.0,22.0,0.0
1,01M019,115.0,135.0,90.0,115.0


In [45]:
# sum up all Fourth Grade Hours
g4 = { 
    'dance': ['DBN'],
    'music': ['DBN'],
    'theater': ['DBN'],
    'visual': ['DBN']
}

for col in arts19_df.columns:
    if ('Q16' in col) & ('R1' in col):
        g4['dance'].append(col)
    elif ('Q16' in col) & ('R2' in col):
        g4['music'].append(col)
    elif ('Q16' in col) & ('R3' in col):
        g4['theater'].append(col)
    elif ('Q16' in col) & ('R4' in col):
        g4['visual'].append(col)
        
arts19_df['G4_Dance_Hrs'] = arts19_df[g4['dance']].fillna(0).sum(axis=1)
arts19_df['G4_Music_Hrs'] = arts19_df[g4['music']].fillna(0).sum(axis=1)
arts19_df['G4_Theater_Hrs'] = arts19_df[g4['theater']].fillna(0).sum(axis=1)
arts19_df['G4_Visual_Hrs'] = arts19_df[g4['visual']].fillna(0).sum(axis=1)

arts19_df[['DBN','G4_Dance_Hrs','G4_Music_Hrs','G4_Theater_Hrs','G4_Visual_Hrs']].head(2)

,DBN,G4_Dance_Hrs,G4_Music_Hrs,G4_Theater_Hrs,G4_Visual_Hrs
0,01M015,0.0,60.0,22.0,0.0
1,01M019,93.0,113.0,120.0,93.0


In [46]:
# sum up all Fifth Grade Hours
g5 = { 
    'dance': ['DBN'],
    'music': ['DBN'],
    'theater': ['DBN'],
    'visual': ['DBN']
}

for col in arts19_df.columns:
    if ('Q17' in col) & ('R1' in col):
        g5['dance'].append(col)
    elif ('Q17' in col) & ('R2' in col):
        g5['music'].append(col)
    elif ('Q17' in col) & ('R3' in col):
        g5['theater'].append(col)
    elif ('Q17' in col) & ('R4' in col):
        g5['visual'].append(col)
        
arts19_df['G5_Dance_Hrs'] = arts19_df[g5['dance']].fillna(0).sum(axis=1)
arts19_df['G5_Music_Hrs'] = arts19_df[g5['music']].fillna(0).sum(axis=1)
arts19_df['G5_Theater_Hrs'] = arts19_df[g5['theater']].fillna(0).sum(axis=1)
arts19_df['G5_Visual_Hrs'] = arts19_df[g5['visual']].fillna(0).sum(axis=1)

arts19_df[['DBN','G5_Dance_Hrs','G5_Music_Hrs','G5_Theater_Hrs','G5_Visual_Hrs']].head(2)

,DBN,G5_Dance_Hrs,G5_Music_Hrs,G5_Theater_Hrs,G5_Visual_Hrs
0,01M015,0.0,60.0,36.0,0.0
1,01M019,73.0,173.0,120.0,73.0


In [58]:
# sum up all Sixth Grade Hours
g6 = { 
    'dance': ['DBN'],
    'music': ['DBN'],
    'theater': ['DBN'],
    'visual': ['DBN']
}

for col in arts19_df.columns:
    if ('Q19' in col) & ('R1' in col):
        g6['dance'].append(col)
    elif ('Q19' in col) & ('R2' in col):
        g6['music'].append(col)
    elif ('Q19' in col) & ('R3' in col):
        g6['theater'].append(col)
    elif ('Q19' in col) & ('R4' in col):
        g6['visual'].append(col)
        
arts19_df['G6_Dance_Hrs'] = arts19_df[g6['dance']].fillna(0).sum(axis=1)
arts19_df['G6_Music_Hrs'] = arts19_df[g6['music']].fillna(0).sum(axis=1)
arts19_df['G6_Theater_Hrs'] = arts19_df[g6['theater']].fillna(0).sum(axis=1)
arts19_df['G6_Visual_Hrs'] = arts19_df[g6['visual']].fillna(0).sum(axis=1)

arts19_df[['DBN','G6_Dance_Hrs','G6_Music_Hrs','G6_Theater_Hrs','G6_Visual_Hrs']].head(2)

,DBN,G6_Dance_Hrs,G6_Music_Hrs,G6_Theater_Hrs,G6_Visual_Hrs
0,01M015,0.0,0.0,0.0,0.0
1,01M019,0.0,0.0,0.0,0.0


In [ ]:
# Todo: Create single loop for all grades
hours_df = pd.DataFrame()
grades = {
    'Q12': 'K',
    'Q13': '1',
    'Q14': '2',
    'Q15': '3',
    'Q16': '4',
    'Q17': '5',
    'Q19': '6'
}
types = {
    'R1': 'Dance',
    'R2': 'Music',
    'R3': 'Theater',
    'R4': 'VisualArts'
}

cols = { 
    'dance': ['DBN'],
    'music': ['DBN'],
    'theater': ['DBN'],
    'visual': ['DBN']
}

for col in arts19_df.columns:
    if ('Q19' in col) & ('R1' in col):
        g6['dance'].append(col)
    elif ('Q19' in col) & ('R2' in col):
        g6['music'].append(col)
    elif ('Q19' in col) & ('R3' in col):
        g6['theater'].append(col)
    elif ('Q19' in col) & ('R4' in col):
        g6['visual'].append(col)
        
arts19_df['G6_Dance_Hrs'] = arts19_df[g6['dance']].fillna(0).sum(axis=1)
arts19_df['G6_Music_Hrs'] = arts19_df[g6['music']].fillna(0).sum(axis=1)
arts19_df['G6_Theater_Hrs'] = arts19_df[g6['theater']].fillna(0).sum(axis=1)
arts19_df['G6_Visual_Hrs'] = arts19_df[g6['visual']].fillna(0).sum(axis=1)

arts19_df[['DBN','G6_Dance_Hrs','G6_Music_Hrs','G6_Theater_Hrs','G6_Visual_Hrs']].head(2)

In [55]:
cols = ['DBN']
for col in arts19_df.columns:
    if ('Q19' in col):
        cols.append(col)
arts19_df[cols].fillna(0)


,DBN,Q19_C1_R1,Q19_C1_R2,Q19_C1_R3,Q19_C1_R4
0,01M015,0.0,0.0,0.0,0.0
1,01M019,0.0,0.0,0.0,0.0
2,01M020,0.0,0.0,0.0,0.0
3,01M034,0.0,0.0,0.0,0.0
4,01M063,0.0,0.0,0.0,0.0
...,...,...,...,...,...
1541,75X596,0.0,0.0,0.0,0.0
1542,75X721,0.0,0.0,0.0,0.0
1543,75X723,15.0,15.0,0.0,0.0
1544,75X754,0.0,0.0,0.0,0.0


In [18]:
sample = 'Q13_C1_R2'
sample[-2:]

'R2'

# load csv files in the data directories
file_dir = '../../mvp/data'
csvFiles = []
[ [csvFiles.append({ 'year': f.split('_')[0].split('-')[1], 'type': dir, 'df': pd.read_csv(os.path.join(file_dir, dir, f), encoding='utf-8') }) for f in os.listdir(os.path.join(file_dir, dir)) if f.endswith('.csv')] for dir in os.listdir('../../mvp/data') if os.path.isdir(os.path.join(file_dir, dir)) ]


arts = {}
test = {}

for o in csvFiles:
    type = o['type']
    year = o['year']
    df = o['df']
    if ( type == 'arts' ):
        df['Year'] = year
        df = df.rename(columns={'Q0_DBN': 'DBN'})
        arts[year] = df
    elif ( type == 'tests' ):
        test[year] = df
    print(f'{year} ({type}) has shape {df.shape}')
    

        

